In [3]:
import openrouteservice
import folium
import pandas as pd
from geopy.distance import geodesic
import json
import requests

def integrated_ev_route_planner(origin, destination, max_range, battery_size, openrouteservice_api_key, openchargemap_api_key):
    client = openrouteservice.Client(key=openrouteservice_api_key)

    def geocode(location):
        try:
            result = client.pelias_search(text=location)
            if 'features' in result and result['features']:
                coordinates = result['features'][0]['geometry']['coordinates']
                return coordinates  # [lon, lat]
            else:
                raise Exception(f"No coordinates found for {location}")
        except Exception as e:
            raise Exception(f"Error geocoding {location}: {str(e)}")

    def calculate_battery_consumed(distance, max_range, battery_size):
        energy_consumption_rate = battery_size / max_range
        consumed_energy = energy_consumption_rate * distance
        percent_consumed_energy = consumed_energy / battery_size * 100
        return percent_consumed_energy

    def find_charging_stations(lat, lon, radius):
        url = f"https://api.openchargemap.io/v3/poi/?output=json&latitude={lat}&longitude={lon}&distance={radius}&distanceunit=KM&maxresults=10&key={openchargemap_api_key}"
        response = requests.get(url)
        if response.status_code == 200:
            stations = response.json()
            return stations
        else:
            print(f"Error fetching charging stations: {response.status_code}")
            return []
        
    def find_stops(route_coordinates, max_range, buffer_range):
        stops = []
        distance_traveled = 0
        current_location = route_coordinates[0]

        for i in range(1, len(route_coordinates)):
            segment_distance = geodesic(current_location[::-1], route_coordinates[i][::-1]).km
            distance_traveled += segment_distance

            if distance_traveled > max_range - buffer_range:
                charging_stations = find_charging_stations(current_location[1], current_location[0], 50)  # 50km radius

                if charging_stations:
                    closest_station = min(charging_stations, key=lambda s: 
                        geodesic((s['AddressInfo']['Latitude'], s['AddressInfo']['Longitude']), current_location[::-1]).km)
                    stops.append({
                        "name": closest_station['AddressInfo']['Title'],
                        "lon": closest_station['AddressInfo']['Longitude'],
                        "lat": closest_station['AddressInfo']['Latitude']
                    })
                    current_location = [closest_station['AddressInfo']['Longitude'], closest_station['AddressInfo']['Latitude']]
                    distance_traveled = 0
                else:
                    print(f"Warning: No charging stations found near {current_location}")

            current_location = route_coordinates[i]

        return stops
    
    def format_duration(minutes):
        if minutes < 60:
            return f"{round(minutes, 1)} mins"
        else:
            hours = int(minutes // 60)
            remaining_minutes = round(minutes % 60)
            return f"{hours} hrs {remaining_minutes} mins"

    def plot_route_on_map(origin_input, origin_coords, destination_input, destination_coords, ev_stations, route):
        route_coordinates = route['geometry']['coordinates']
        
        map_center = [(origin_coords[1] + destination_coords[1]) / 2, (origin_coords[0] + destination_coords[0]) / 2]
        map_obj = folium.Map(location=map_center[::-1], zoom_start=12, width=400, height=700)

        lats = [coord[1] for coord in route_coordinates] + [origin_coords[1], destination_coords[1]]
        lons = [coord[0] for coord in route_coordinates] + [origin_coords[0], destination_coords[0]]
        min_lat, max_lat = min(lats), max(lats)
        min_lon, max_lon = min(lons), max(lons)

        map_obj.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
        
        folium.Marker(location=origin_coords[::-1], popup=f'Origin: {origin_input}', icon=folium.Icon(color='green',icon='home', prefix='fa')).add_to(map_obj)
        folium.Marker(location=destination_coords[::-1], popup=f'Destination: {destination_input}', icon=folium.Icon(color='blue',icon='location-pin', prefix='fa')).add_to(map_obj)
        
        for stop in ev_stations:
            folium.Marker(
                location = [stop['lat'], stop['lon']],
                popup = stop['name'],
                icon = folium.Icon(color='red',icon='charging-station', prefix='fa')
            ).add_to(map_obj)
        
        folium.PolyLine(locations=[coord[::-1] for coord in route_coordinates], color='blue', weight=3, opacity=0.7).add_to(map_obj)

        properties = route.get('properties', {})
        segments = properties.get('segments', [])
        
        table_data = []
        locations = [f'{origin_input} (Origin)'] + [station['name'] for station in ev_stations] + [f'{destination_input} (Destination)']
        
        cumulative_distance = 0
        cumulative_duration = 0
        
        for i, location in enumerate(locations):
            if i > 0 and i-1 < len(segments):
                segment = segments[i-1]
                segment_distance = segment.get('distance', 0) / 1000  # Convert to km
                segment_duration = segment.get('duration', 0) / 60  # Convert to minutes
                cumulative_distance += segment_distance
                cumulative_duration += segment_duration
            
            table_data.append({
                'Point': location,
                'Cum. Distance (km)': round(cumulative_distance, 2),
                'Cum. Duration (mins)': format_duration(cumulative_duration)
            })
        
        df = pd.DataFrame(table_data)
        
        return map_obj, df, cumulative_distance, cumulative_duration

    # Main execution
    try:
        origin_coords = geocode(origin)
        destination_coords = geocode(destination)

        # Get the direct route first
        direct_route_params = {
            'coordinates': [origin_coords, destination_coords],
            'profile': 'driving-car',
            'format': 'geojson'
        }
        direct_route_result = client.directions(**direct_route_params)

        if not isinstance(direct_route_result, dict) or 'features' not in direct_route_result or not direct_route_result['features']:
            raise Exception(f"Unexpected direct route result format: {type(direct_route_result)}")
        
        direct_route = direct_route_result['features'][0]
        direct_properties = direct_route.get('properties', {})
        direct_summary = direct_properties.get('summary', {})
        direct_distance = direct_summary.get('distance', 0) / 1000  # Convert to km
        direct_duration = direct_summary.get('duration', 0) / 60  # Convert to minutes

        estimated_battery_consumption = calculate_battery_consumed(direct_distance, max_range, battery_size)

        print(f"Direct route distance: {direct_distance:.2f} km")
        print(f"Direct route duration: {format_duration(direct_duration)}")
        print(f"Vehicle range: {max_range} km")
        print(f"Battery size: {battery_size} kWh")
        print(f"Estimated battery consumption for direct route: {estimated_battery_consumption:.2f}%")

        # Find EV chargers along the direct route
        ev_stations = find_stops(direct_route['geometry']['coordinates'], max_range, 50)  # 50 km buffer
        print(f"Found {len(ev_stations)} charging stops")

        if ev_stations:
            waypoints = [origin_coords] + [[stop['lon'], stop['lat']] for stop in ev_stations] + [destination_coords]
            route_with_stops_params = {
                'coordinates': waypoints,
                'profile': 'driving-car',
                'format': 'geojson'
            }
            route_with_stops_result = client.directions(**route_with_stops_params)

            if not isinstance(route_with_stops_result, dict) or 'features' not in route_with_stops_result or not route_with_stops_result['features']:
                raise Exception(f"Unexpected route with stops result format: {type(route_with_stops_result)}")
            route_with_stops = route_with_stops_result['features'][0]
        else:
            route_with_stops = direct_route

        # Plot the route
        route_map, route_data, total_distance, total_duration = plot_route_on_map(origin, origin_coords, destination, destination_coords, ev_stations, route_with_stops)

        # Consider distance will be longer if there are stops on the route
        total_battery_consumed = calculate_battery_consumed(total_distance, max_range, battery_size)

        print(f"Total distance: {total_distance:.2f} km")
        print(f"Total battery comsuption Estimation: {total_battery_consumed:.2f}%")
        print(f"Total duration: {format_duration(total_duration)}")
        print("\nRoute details:")
        print(route_data)

        return route_map, route_data, direct_distance, total_distance, total_battery_consumed

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None, 0, 0, 0


In [4]:
# Example usage:
origin = "Shepparton"
destination = "Melbourne"
max_range = 200  # km 
battery_size = 75  # kWh
openrouteservice_api_key = "5b3ce3597851110001cf624842d6ce02dd3743baa1061059094207f5" # personal key
openchargemap_api_key = "a421d63d-e002-4fee-9fe3-76a9836705b3"  # personal key

route_map, route_data, direct_distance, total_distance, total_battery_consumed = integrated_ev_route_planner(
    origin, destination, max_range, battery_size, openrouteservice_api_key, openchargemap_api_key 
)

route_map

Direct route distance: 185.39 km
Direct route duration: 2 hrs 23 mins
Vehicle range: 200 km
Battery size: 75 kWh
Estimated battery consumption for direct route: 92.69%
Found 1 charging stops
Total distance: 201.94 km
Total battery comsuption Estimation: 100.97%
Total duration: 2 hrs 40 mins

Route details:
                     Point  Cum. Distance (km) Cum. Duration (mins)
0      Shepparton (Origin)                0.00               0 mins
1            Club Trillium              155.50        1 hrs 52 mins
2  Melbourne (Destination)              201.94        2 hrs 40 mins
